In [131]:
from pyspark.sql import SparkSession, functions, types

spark = SparkSession\
        .builder\
        .appName("SparkRatings")\
        .getOrCreate()
sc = spark.sparkContext

# ratings = spark.read.csv("hdfs:///bigdata/books/BX-Book-Ratings_sample.csv", header=True)
# users = spark.read.csv("hdfs:///bigdata/books/BX-Users_sample.csv", header=True)


# omasta kansiosta
ratings = spark.read.csv("BX-Book-Ratings.csv", header=True, sep=";")
users = spark.read.csv("BX-Users.csv", header=True, sep=";")

In [132]:
split_col = functions.split(users['Location'], ',')
users = users.withColumn('Country', split_col.getItem(2))

In [133]:
users = users.drop('Location', 'Age')

In [134]:
ratings = ratings.drop('ISBN')

In [137]:
inner_join = users.join(ratings, ['User-ID'])

In [138]:
df = df.withColumn("Book-Rating", df["Book-Rating"].cast(types.IntegerType()))

In [139]:
avg = df.groupBy('Country').avg('Book-Rating')

In [140]:
count = df.groupBy("Country").count()

In [141]:
inner_join2 = count.join(avg, ['Country'])

In [142]:
data = inner_join2.select(inner_join2["Country"].alias("country"), inner_join2["count"].alias("ratings"), inner_join2["avg(Book-Rating)"].alias("average rating"))

In [143]:
data = data.withColumn("average rating", functions.round(data["average rating"], 1))
data.show(5)

+--------------------+-------+--------------+
|             country|ratings|average rating|
+--------------------+-------+--------------+
|           la suisse|      3|           6.0|
|  baden-wuerttemberg|      2|           0.0|
|         west indies|     12|           3.7|
|            missouri|      5|           4.2|
| netherlands anti...|      3|           6.0|
+--------------------+-------+--------------+
only showing top 5 rows



In [144]:
csvdata = data.select(functions.concat(data["country"], functions.lit(";"), data["ratings"], functions.lit(";"), data["average rating"]))
csvdata = csvdata.withColumnRenamed("concat(country, ;, ratings, ;, average rating)", "csv")
csvdata.show()

+--------------------+
|                 csv|
+--------------------+
|     la suisse;3;6.0|
| baden-wuerttembe...|
|  west indies;12;3.7|
|      missouri;5;4.2|
| netherlands anti...|
|      unknown;1;10.0|
|        61 men;1;0.0|
|   new mexico;11;4.4|
| republic of pana...|
|     worcester;1;8.0|
|       egypt;102;3.0|
| north carolina;1...|
|  south island;2;0.0|
|       jamaica;9;2.7|
|        veneto;1;8.0|
|    burlington;1;0.0|
| canary islands;1...|
|        hawaii;5;3.8|
|     middlesex;1;8.0|
|     fort bend;3;7.7|
+--------------------+
only showing top 20 rows

